# **Cuaderno de ETL: De Datos Abiertos a un Modelo de Estrella**

**Objetivo:** Tomar un conjunto de datos públicos sobre estadísticas de educación en Colombia y transformarlo en un modelo dimensional de estrella, listo para ser analizado con herramientas de Business Intelligence.

**Fuente de Datos:** [MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev/about_data)

**Nuestro Modelo de Estrella a Construir:**

* **Tabla de Hechos (Fact_Matriculas):**
    * `id_tiempo` (FK)
    * `id_geografia` (FK)
    * `total_matriculados` (Métrica)
    ...

* **Tablas de Dimensiones:**
    * `Dim_Tiempo` (año)
    * `Dim_Geografia` (departamento, municipio)

¡Manos a la obra!

In [13]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [14]:
df_raw

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,...,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


## **3. Transformación (Transform)**

Esta es la fase más importante. Aquí limpiamos los datos crudos y los moldeamos para que encajen en nuestro modelo de estrella.

**Pasos:**
1.  **Limpieza y Preparación:** Convertiremos las columnas a los tipos de datos correctos y manejaremos valores faltantes. La columna `matricula` es nuestra métrica principal.
2.  **Creación de Dimensiones:** A partir del DataFrame limpio, crearemos una tabla (DataFrame) para cada dimensión, asegurándonos de que no tengan filas duplicadas y asignando una **llave subrogada** (un ID numérico único).
3.  **Creación de la Tabla de Hechos:** Construiremos la tabla de hechos, que contendrá nuestra métrica (`total_matriculados`) y las llaves foráneas que la conectan a cada dimensión.

## Solución 

1. *Limpieza y Preparación*

In [15]:
import pandas as pd

In [16]:
df_raw.columns = df_raw.columns.str.lower()

In [17]:
print(df_raw.columns.tolist())

['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento', 'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n', 'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria', 'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet']


In [18]:
df_raw.isnull().sum()

a_o                               0
c_digo_municipio                  0
municipio                         0
c_digo_departamento               0
departamento                      0
c_digo_etc                        0
etc                               0
poblaci_n_5_16                    6
tasa_matriculaci_n_5_16         115
cobertura_neta                  111
cobertura_neta_transici_n        52
cobertura_neta_primaria          91
cobertura_neta_secundaria        94
cobertura_neta_media             93
cobertura_bruta                  68
cobertura_bruta_transici_n       97
cobertura_bruta_primaria         81
cobertura_bruta_secundaria       88
cobertura_bruta_media           127
deserci_n                       142
deserci_n_transici_n            903
deserci_n_primaria              242
deserci_n_secundaria            270
deserci_n_media                 734
aprobaci_n                       25
aprobaci_n_transici_n            93
aprobaci_n_primaria              25
aprobaci_n_secundaria       

In [19]:
print(df_raw.dtypes)

a_o                            object
c_digo_municipio               object
municipio                      object
c_digo_departamento            object
departamento                   object
c_digo_etc                     object
etc                            object
poblaci_n_5_16                 object
tasa_matriculaci_n_5_16        object
cobertura_neta                 object
cobertura_neta_transici_n      object
cobertura_neta_primaria        object
cobertura_neta_secundaria      object
cobertura_neta_media           object
cobertura_bruta                object
cobertura_bruta_transici_n     object
cobertura_bruta_primaria       object
cobertura_bruta_secundaria     object
cobertura_bruta_media          object
deserci_n                      object
deserci_n_transici_n           object
deserci_n_primaria             object
deserci_n_secundaria           object
deserci_n_media                object
aprobaci_n                     object
aprobaci_n_transici_n          object
aprobaci_n_p

In [20]:

columnas_numericas = [
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta',
    'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria',
    'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n',
    'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media',
    'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria',
    'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria',
    'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n',
    'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media',
    'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
    'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet'
]

# Convertir esas columnas a tipo numérico
for col in columnas_numericas:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')


In [21]:

#Se relleno los datos nulos con la media para mantener la estructura de la base de datos 
df_raw[columnas_numericas] = df_raw[columnas_numericas].fillna(df_raw[columnas_numericas].mean())


In [22]:
print("Años únicos:", df_raw['a_o'].unique())
print("Departamentos únicos:", df_raw['departamento'].nunique())
print("Municipios únicos:", df_raw['municipio'].nunique())

Años únicos: ['2023' '2022' '2021' '2020' '2019' '2018' '2017' '2016' '2015' '2014'
 '2013' '2012' '2011']
Departamentos únicos: 36
Municipios únicos: 1037


In [23]:

dept_check = df_raw[['c_digo_departamento', 'departamento']].drop_duplicates()
dept_group = dept_check.groupby('c_digo_departamento').agg({'departamento': pd.Series.nunique})
conflictos = dept_group[dept_group['departamento'] > 1]

print(conflictos)

# Error nombre
nombres_conflictivos = df_raw[df_raw['c_digo_departamento'].isin(conflictos.index)][['c_digo_departamento', 'departamento']].drop_duplicates()
print(nombres_conflictivos)

                     departamento
c_digo_departamento              
11                              2
88                              2
     c_digo_departamento                                       departamento
32                    88  Archipiélago de San Andrés, Providencia y Sant...
974                   11                                       Bogotá, D.C.
3389                  88  Archipiélago de San Andrés. Providencia y Sant...
3653                  11                                        Bogotá D.C.


In [24]:
conflictos_municipio = (
    df_raw.groupby('c_digo_municipio')[['municipio', 'c_digo_departamento']]
    .nunique()
    .query('municipio > 1 or c_digo_departamento > 1')
)

print(conflictos_municipio)

                  municipio  c_digo_departamento
c_digo_municipio                                
11001                     2                    1


In [25]:
df_raw['departamento'] = df_raw['departamento'].replace({
    'Bogotá D.C.': 'Bogotá, D.C.',
    'Archipiélago de San Andrés. Providencia y Santa Catalina.': 'Archipiélago de San Andrés, Providencia y Santa Catalina'
})

In [26]:
df_raw.loc[df_raw['c_digo_departamento'] == 88, 'departamento'] = \
    "Archipiélago de San Andrés, Providencia y Santa Catalina"

In [27]:
df_raw['departamento'] = df_raw['departamento'].str.strip().str.replace(r'[.,;]+', '', regex=True)

In [28]:
df_raw['municipio'] = df_raw['municipio'].str.strip().str.replace(r'[.,]', '', regex=True)


In [29]:
df_raw = df_raw[df_raw['departamento'] != 'NACIONAL']
df_raw = df_raw[~df_raw['departamento'].str.lower().str.contains('NACIONAL', na=False)]


Cargue Divipola

In [30]:
import pandas as pd

url = "https://www.datos.gov.co/api/views/gdxc-w37w/rows.csv?accessType=DOWNLOAD"
divipola = pd.read_csv(url)

divipola.head()


,Código Departamento,Nombre Departamento,Código Municipio,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada,longitud,Latitud
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio,"-75,141346","6,376061"
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio,"-75,702188","6,038708"


In [31]:
print(divipola.columns)


Index(['Código Departamento', 'Nombre Departamento', 'Código Municipio',
       'Nombre Municipio', 'Tipo: Municipio / Isla / Área no municipalizada',
       'longitud', 'Latitud'],
      dtype='object')


In [32]:
divipola = divipola.rename(columns={
    'Código Departamento': 'c_digo_departamento',
    'Nombre Departamento': 'departamento',
    'Código Municipio': 'c_digo_municipio',
    'Nombre Municipio': 'municipio',
    'Tipo: Municipio / Isla / Área no municipalizada': 'tipo_territorial',
    'longitud': 'longitud',
    'Latitud': 'latitud'
})

2. *Creación de Dimensiones* 

In [33]:
# Convertir a numérico forzando errores como NaN
for col in ['c_digo_departamento', 'c_digo_municipio', 'c_digo_etc']:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

# Eliminar filas con valores faltantes
df_raw = df_raw.dropna(subset=['c_digo_departamento', 'c_digo_municipio', 'c_digo_etc'])

# Convertir a enteros
df_raw['c_digo_departamento'] = df_raw['c_digo_departamento'].astype(int)
df_raw['c_digo_municipio'] = df_raw['c_digo_municipio'].astype(int)
df_raw['c_digo_etc'] = df_raw['c_digo_etc'].astype(int)


In [34]:
dim_departamento = df_raw[['c_digo_departamento', 'departamento']].drop_duplicates()

dim_departamento = dim_departamento.merge(
    divipola[['c_digo_departamento', 'departamento']].drop_duplicates(),
    on='c_digo_departamento', how='left', suffixes=('', '_divipola')
)

dim_departamento = dim_departamento.reset_index(drop=True)
dim_departamento['id_departamento'] = dim_departamento.index + 1
dim_departamento = dim_departamento[['id_departamento', 'c_digo_departamento', 'departamento']]

# Validación
assert dim_departamento['c_digo_departamento'].is_unique, "Claves duplicadas en dim_departamento"
print("dim_departamento generada correctamente.")



dim_departamento generada correctamente.


In [35]:

dim_municipio = df_raw[['c_digo_municipio', 'c_digo_departamento']].drop_duplicates()

dim_municipio = dim_municipio.merge(
    divipola[['c_digo_municipio', 'municipio', 'tipo_territorial', 'latitud', 'longitud']],
    on='c_digo_municipio', how='left'
)

dim_municipio = dim_municipio.reset_index(drop=True)
dim_municipio['id_municipio'] = dim_municipio.index + 1

dim_municipio = dim_municipio[['id_municipio', 'c_digo_municipio', 'c_digo_departamento',
                               'municipio', 'tipo_territorial', 'latitud', 'longitud']]

# Validación
assert dim_municipio['c_digo_municipio'].is_unique, "Claves duplicadas en dim_municipio"
print("dim_municipio generada correctamente.")


dim_municipio generada correctamente.


In [36]:
dim_tiempo = df_raw[['a_o']].drop_duplicates().reset_index(drop=True)
dim_tiempo['id_tiempo'] = dim_tiempo.index + 1
dim_tiempo = dim_tiempo[['id_tiempo', 'a_o']]

# Validación
assert dim_tiempo['a_o'].is_unique, "Años duplicados en dim_tiempo"
print("dim_tiempo generada correctamente.")



dim_tiempo generada correctamente.


3. *Creación de la tabla de Hechos*

In [37]:
df_hechos = df_raw.copy()

df_hechos = df_hechos.merge(
    dim_departamento[['id_departamento', 'c_digo_departamento']],
    on='c_digo_departamento', how='left'
)

df_hechos = df_hechos.merge(
    dim_municipio[['id_municipio', 'c_digo_municipio']],
    on='c_digo_municipio', how='left'
)

df_hechos = df_hechos.merge(
    dim_tiempo[['id_tiempo', 'a_o']],
    on='a_o', how='left'
)


In [38]:
columnas_medidas = [
    'tasa_matriculaci_n_5_16', 'cobertura_neta',
    'reprobaci_n_primaria', 'reprobaci_n_secundaria',
    'reprobaci_n_media', 'repitencia'
]

hechos_final = df_hechos[['id_departamento', 'id_municipio', 'id_tiempo'] + columnas_medidas]

print(" Tabla de hechos generada")


 Tabla de hechos generada


In [39]:
hechos_final.isnull().sum()

id_departamento            0
id_municipio               0
id_tiempo                  0
tasa_matriculaci_n_5_16    0
cobertura_neta             0
reprobaci_n_primaria       0
reprobaci_n_secundaria     0
reprobaci_n_media          0
repitencia                 0
dtype: int64

In [40]:
hechos_final.describe()


,id_departamento,id_municipio,id_tiempo,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia
count,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000
mean,18.612467,561.346249,7.001234,84.989221,85.570607,3.885558,6.810386,4.131648,3.300150
std,9.416219,323.815817,3.741511,18.478286,16.825781,3.789782,6.130932,4.190801,3.353227
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12.000000,281.000000,4.000000,74.950000,76.962500,0.430000,0.762500,0.682500,0.730000
50%,20.000000,561.000000,7.000000,85.200000,86.280000,3.380000,6.220000,3.160000,2.190000
75%,26.000000,842.000000,10.000000,95.307500,94.477500,5.980000,10.790000,6.360000,5.047500
max,33.000000,1122.000000,13.000000,279.030000,264.540000,51.970000,76.970000,67.860000,37.470000


In [41]:

hechos_final.head()

,id_departamento,id_municipio,id_tiempo,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia
0,1,1,1,62.62,62.62,1.96,16.51,2.04,9.52
1,2,2,1,53.27,53.27,7.11,9.39,1.75,9.34
2,2,3,1,32.52,32.52,6.93,14.13,7.81,8.65
3,3,4,1,59.57,59.57,4.04,8.33,4.60,16.18
4,3,5,1,51.30,51.30,7.32,15.28,7.27,9.24


In [42]:
hechos_nombres = hechos_final.copy()

hechos_nombres = hechos_nombres.merge(
    dim_municipio[['id_municipio', 'municipio']],
    on='id_municipio', how='left'
)

hechos_nombres = hechos_nombres.merge(
    dim_departamento[['id_departamento', 'departamento']],
    on='id_departamento', how='left'
)


hechos_nombres = hechos_nombres.merge(
    dim_tiempo[['id_tiempo', 'a_o']],  
    on='id_tiempo', how='left'
)


In [43]:
hechos_nombres.head()

,id_departamento,id_municipio,id_tiempo,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,municipio,departamento,a_o
0,1,1,1,62.62,62.62,1.96,16.51,2.04,9.52,ABRIAQUÍ,Antioquia,2023
1,2,2,1,53.27,53.27,7.11,9.39,1.75,9.34,EL RETORNO,Guaviare,2023
2,2,3,1,32.52,32.52,6.93,14.13,7.81,8.65,MIRAFLORES,Guaviare,2023
3,3,4,1,59.57,59.57,4.04,8.33,4.60,16.18,MITÚ,Vaupés,2023
4,3,5,1,51.30,51.30,7.32,15.28,7.27,9.24,CARURÚ,Vaupés,2023


## Preguntas

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

2. ¿Cómo compararía el rendimiento educativo por municipios?

3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

Este ejercicio se entrega en un archivo Jupyter Notebook (.ipynb) que contenga el código necesario para realizar las consultas en SQL y que previamente haya creado la bodega de datos con un modelo dimensional adecuado.

####  Porcentaje de escolaridad por municipio

In [ ]:

porcentaje_escolaridad = hechos_nombres.groupby(['departamento', 'municipio'])['tasa_matriculaci_n_5_16'].mean().reset_index()
porcentaje_escolaridad = porcentaje_escolaridad.rename(columns={'tasa_matriculaci_n_5_16': 'porcentaje_escolaridad'})
porcentaje_escolaridad = porcentaje_escolaridad.sort_values(by='porcentaje_escolaridad', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#porcentaje_escolaridad -- Muestra toda la tabla
porcentaje_escolaridad.head(15)



,departamento,municipio,porcentaje_escolaridad
517,Cundinamarca,COTA,161.882281
158,Atlántico,PUERTO COLOMBIA,153.048434
545,Cundinamarca,LA CALERA,144.078462
894,Quindio,SALENTO,139.272308
592,Cundinamarca,TENJO,134.685061
528,Cundinamarca,FÚQUENE,128.590769
584,Cundinamarca,SUBACHOQUE,127.242308
970,Santander,PUENTE NACIONAL,125.726923
601,Cundinamarca,VENECIA,124.103077
451,Cesar,EL PASO,122.451538


#### Rendimiento educativo por municipio

In [48]:
variables_rendimiento = [
    'repitencia', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media'
]
rendimiento = hechos_nombres.groupby(['departamento', 'municipio'])[variables_rendimiento].mean().reset_index()
rendimiento['rendimiento_promedio'] = rendimiento[variables_rendimiento].mean(axis=1)
rendimiento_ordenado = rendimiento.sort_values(by='rendimiento_promedio', ascending=True)

print(rendimiento_ordenado[['departamento', 'municipio', 'rendimiento_promedio']].head(15))


     departamento      municipio  rendimiento_promedio
575  Cundinamarca  SAN FRANCISCO              0.363846
588  Cundinamarca      SUTATAUSA              0.466538
594  Cundinamarca       TIBIRITA              0.491923
547  Cundinamarca       LA PALMA              0.608846
275        Boyacá        OTANCHE              0.801154
566  Cundinamarca  PUERTO SALGAR              0.820577
563  Cundinamarca          PANDI              0.854615
909     Santander         AGUADA              0.879038
506  Cundinamarca        CABRERA              0.940769
597  Cundinamarca        TOPAIPÍ              0.961923
778        Nariño         CUMBAL              0.989231
539  Cundinamarca      GUATAVITA              0.995577
222        Boyacá         BOYACÁ              0.997692
813        Nariño       PUPIALES              1.032500
580  Cundinamarca       SILVANIA              1.039423


#### Departamentos con mejor cobertura educativa

In [49]:
!pip install pandasql

In [53]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

consulta_sql = """
SELECT departamento, AVG(cobertura_neta) AS cobertura_promedio
FROM hechos_nombres
GROUP BY departamento
ORDER BY cobertura_promedio DESC
"""

resultado_sql = pysqldf(consulta_sql)
print(resultado_sql)

                                         departamento  cobertura_promedio
0                                             Quindio           94.579615
1                                               Sucre           93.849172
2                                               Cesar           93.730656
3                                           Magdalena           93.263854
4                                                Meta           90.630439
5                                           Bogotá DC           89.493077
6                                              Tolima           89.239083
7                                        Cundinamarca           88.541257
8                                            Casanare           88.517623
9                                           Antioquia           88.095366
10                                            Córdoba           88.032417
11                                          Atlántico           87.567434
12                                    